# 20_Gold: サインイン分析マートの作成

## 概要
このノートブックでは、Silver層のデータから、ビジネスユーザーやダッシュボードで直接利用できる集計済みのマートテーブルを作成します。

## 処理フロー
1. 日次サマリーテーブル
2. 失敗理由別の集計テーブル
3. アプリケーション別の集計テーブル
4. Impossible Travel明細テーブル
5. ユーザー別リスクスコアテーブル
6. ダッシュボード用ビュー

## メダリオンアーキテクチャ - Gold層
- **目的**: ビジネスユーザーが直接利用できる、高度に集計・加工されたデータを提供
- **特徴**: 特定のユースケースに最適化、事前集計、ビジネスメトリクスの計算
- **利点**: クエリが高速、ビジネスロジックが明確、レポート作成が容易

## セキュリティダッシュボードの設計
これらのマートは、以下のようなセキュリティ監視ダッシュボードに利用できます：
- **トレンド分析**: 日次のサインイン成功/失敗率
- **異常検知**: 失敗理由の急増、Impossible Travelの発生
- **リスク評価**: ユーザーごとのリスクスコア、高リスクユーザーの特定
- **アプリケーション監視**: アプリケーション別のサインイン状況


## Step 1: 日次サマリーテーブル

日付ごとの基本的なサインインメトリクスを集計します。

### 含まれる指標
- `total_signins`: 総サインイン数
- `success_count`: 成功したサインイン数
- `failure_count`: 失敗したサインイン数
- `unique_users`: ユニークユーザー数

**用途**: トレンド分析、日次レポート


In [0]:
USE CATALOG users;
USE users.yukiteru_koide;

-- 1) 日次サマリー
CREATE OR REPLACE TABLE users.yukiteru_koide.signin_daily_summary AS
SELECT
  event_date,
  COUNT(*) AS total_signins,
  SUM(CASE WHEN is_success THEN 1 ELSE 0 END) AS success_count,
  SUM(CASE WHEN is_failure THEN 1 ELSE 0 END) AS failure_count,
  COUNT(DISTINCT userId) AS unique_users
FROM users.yukiteru_koide.signin_events_silver
GROUP BY event_date;


## Step 2: 失敗理由別テーブル

日付と失敗理由ごとの失敗数を集計します。

### 含まれる指標
- `event_date`: 日付
- `failure_reason`: 失敗理由（NULL の場合は "Unknown"）
- `failures`: 失敗回数

**用途**: 失敗の根本原因分析、トラブルシューティング


In [0]:
-- 2) 失敗理由 × 日
CREATE OR REPLACE TABLE users.yukiteru_koide.signin_fail_by_reason_day AS
SELECT
  event_date,
  coalesce(statusFailureReason, 'Unknown') AS failure_reason,
  COUNT(*) AS failures
FROM users.yukiteru_koide.signin_events_silver
WHERE is_failure
GROUP BY event_date, coalesce(statusFailureReason, 'Unknown');


## Step 3: アプリケーション別テーブル

日付とアプリケーションごとのサインイン成功/失敗を集計します。

### 含まれる指標
- `event_date`: 日付
- `appDisplayName`: アプリケーション名
- `success_count`: 成功回数
- `failure_count`: 失敗回数

**用途**: アプリケーション別のアクセス状況監視、問題のあるアプリの特定


In [0]:
-- 3) アプリ別 × 日
CREATE OR REPLACE TABLE users.yukiteru_koide.signin_by_app_day AS
SELECT
  event_date,
  appDisplayName,
  SUM(CASE WHEN is_success THEN 1 ELSE 0 END) AS success_count,
  SUM(CASE WHEN is_failure THEN 1 ELSE 0 END) AS failure_count
FROM users.yukiteru_koide.signin_events_silver
GROUP BY event_date, appDisplayName;


## Step 4: Impossible Travel明細テーブル

物理的に不可能な移動が検出されたサインインイベントの明細を抽出します。

### 含まれる情報
- タイムスタンプ、ユーザー情報
- 現在と前回の国/都市
- デバイス情報（OS、ブラウザ）
- ネットワーク情報（IPアドレス、クライアントアプリ）

**用途**: セキュリティインシデントの調査、なりすましアクセスの検出


In [0]:
-- 4) Impossible Travel 明細
CREATE OR REPLACE TABLE users.yukiteru_koide.signin_geo_impossible_travel AS
SELECT
  created_ts, userPrincipalName, userId,
  locationCountryOrRegion, locationCity,
  prev_country, prev_city,
  impossible_travel,
  ipAddress, clientAppUsed, deviceOperatingSystem, browser
FROM users.yukiteru_koide.signin_events_silver
WHERE impossible_travel = true;


## Step 5: ユーザー別リスクスコアテーブル

ユーザーごとに日次のリスクスコアを計算します。

### リスクスコアの計算ロジック
各セキュリティイベントに重み付けしてスコアを算出：
- **ログイン失敗**: 1点 × 失敗回数
- **営業時間外の成功ログイン**: 2点 × 回数
- **条件付きアクセス失敗**: 2点 × 回数
- **Impossible Travel**: 5点 × 回数

高いスコアほど、セキュリティリスクが高いことを示します。

**用途**: 高リスクユーザーの特定、セキュリティチームへのアラート、優先的な調査対象の決定


In [0]:
-- 5) ユーザー別リスク（日次）
CREATE OR REPLACE TABLE users.yukiteru_koide.signin_risk_by_user_day AS
SELECT
  userId,
  ANY_VALUE(userPrincipalName) AS userPrincipalName,
  event_date,
  SUM(CASE WHEN is_failure THEN 1 ELSE 0 END) AS failed_logins,
  SUM(CASE WHEN is_off_hours AND is_success THEN 1 ELSE 0 END) AS offhours_success,
  SUM(CASE WHEN ca_failed THEN 1 ELSE 0 END) AS ca_failures,
  SUM(CASE WHEN impossible_travel THEN 1 ELSE 0 END) AS impossible_travels,
  (SUM(CASE WHEN is_failure THEN 1 ELSE 0 END) * 1)
  + (SUM(CASE WHEN is_off_hours AND is_success THEN 1 ELSE 0 END) * 2)
  + (SUM(CASE WHEN ca_failed THEN 1 ELSE 0 END) * 2)
  + (SUM(CASE WHEN impossible_travel THEN 1 ELSE 0 END) * 5) AS risk_score
FROM users.yukiteru_koide.signin_events_silver
GROUP BY userId, event_date;


## Step 6: ダッシュボード用ビュー

ダッシュボードのKPIカードで使用する、直近7日間のサマリー指標を提供するビューを作成します。

### 提供する指標
- `success_7d`: 過去7日間の成功ログイン数
- `failure_7d`: 過去7日間の失敗ログイン数
- `impossible_travel_7d`: 過去7日間のImpossible Travel検出数

**用途**: エグゼクティブダッシュボード、セキュリティサマリーレポート


In [0]:
-- ダッシュボード用ビュー
CREATE OR REPLACE VIEW users.yukiteru_koide.v_dashboard_cards AS
SELECT
  (SELECT SUM(success_count) FROM users.yukiteru_koide.signin_daily_summary
   WHERE event_date >= date_sub(current_date(), 7)) AS success_7d,
  (SELECT SUM(failure_count) FROM users.yukiteru_koide.signin_daily_summary
   WHERE event_date >= date_sub(current_date(), 7)) AS failure_7d,
  (SELECT COUNT(*) FROM users.yukiteru_koide.signin_geo_impossible_travel
   WHERE created_ts >= date_sub(current_date(), 7)) AS impossible_travel_7d;


## Step 7: 全テーブルの最適化

作成した全てのGoldマートテーブルを最適化し、クエリパフォーマンスを最大化します。


In [0]:
OPTIMIZE users.yukiteru_koide.signin_daily_summary;
OPTIMIZE users.yukiteru_koide.signin_fail_by_reason_day;
OPTIMIZE users.yukiteru_koide.signin_by_app_day;
OPTIMIZE users.yukiteru_koide.signin_geo_impossible_travel;
OPTIMIZE users.yukiteru_koide.signin_risk_by_user_day;
